## Feature creation after Merging all Datasets &  EDA Bike Movements

In [2]:
# Load libraries 
import time
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [3]:
# Load files
full_dataset = "add_path_to_merged_dataset"

labeled_bike_users_df = pd.read_csv(full_dataset, header = 0, sep = ';')

In [4]:
labeled_bike_users_df.head()

,_id,user_day_code,idplug_base,user_type,idunplug_base,travel_time,idunplug_station,ageRange,idplug_station,unplug_hourTime,zip_code,temp_unplug_hourTime,year,month,day,hour,travel_time_min,travel_revenue,id_weather,80,81,82,83,86,87,88,89,date,week_day,WorkingDay_Holiday_Sunday,holiday_type,holiday_desc,day_of_week,unplug_station_tot_bases,idunplug_latitude,idunplug_longitude,plug_station_tot_bases,idplug_latitude,idplug_longitude,mh_distance_meters,time_group,unique_user_day_code,tot_trips_day
0,5c4b07ea2f38432e007daaba,a18550d468153bb8e1808d4f0a85955579951832772996...,10,1,1,415,133,5,36,2019-01-01T00:00:00.000+0100,28005,2019-01-01 00:00:00+01:00,2019,1,1,0,6.916667,0.5,24,6.0,0.69,22.0,4.4,67.0,957.0,101.0,0.0,2019-01-01,tuesday,holiday,national_holiday,Año Nuevo,1,24,40.403408,-3.710811,18,40.415010,-3.706103,1760.389172,0,a18550d468153bb8e1808d4f0a85955579951832772996...,1
1,5c4b07ea2f38432e007daabd,5922b4a1a2591161ebf1ad66f42f52cc7796f3754c2c76...,18,1,6,417,105,3,155,2019-01-01T00:00:00.000+0100,28009,2019-01-01 00:00:00+01:00,2019,1,1,0,6.950000,0.5,24,6.0,0.69,22.0,4.4,67.0,957.0,101.0,0.0,2019-01-01,tuesday,holiday,national_holiday,Año Nuevo,1,24,40.424846,-3.673863,18,40.444226,-3.678717,2641.794256,0,5922b4a1a2591161ebf1ad66f42f52cc7796f3754c2c76...,1
2,5c4b07ea2f38432e007daabe,ed1c1a22c120e7d42cc522c33ab41db549791f3914b273...,4,1,12,448,51,4,44,2019-01-01T00:00:00.000+0100,28005,2019-01-01 00:00:00+01:00,2019,1,1,0,7.466667,0.5,24,6.0,0.69,22.0,4.4,67.0,957.0,101.0,0.0,2019-01-01,tuesday,holiday,national_holiday,Año Nuevo,1,24,40.404785,-3.702827,24,40.413885,-3.704941,1221.373443,0,ed1c1a22c120e7d42cc522c33ab41db549791f3914b273...,1
3,5c4b07ea2f38432e007daac2,b70eb1a71e53c6351b4835f618a459886e2045a44635e4...,17,1,11,468,100,5,171,2019-01-01T00:00:00.000+0100,28028,2019-01-01 00:00:00+01:00,2019,1,1,0,7.800000,0.5,24,6.0,0.69,22.0,4.4,67.0,957.0,101.0,0.0,2019-01-01,tuesday,holiday,national_holiday,Año Nuevo,1,24,40.422658,-3.687055,24,40.429887,-3.671282,2405.058377,0,b70eb1a71e53c6351b4835f618a459886e2045a44635e4...,1
4,5c4b07ea2f38432e007daac4,1ad9b018315170d824bb396889046eaeff8ce0a0be46b3...,13,1,2,149,67,4,65,2019-01-01T00:00:00.000+0100,28009,2019-01-01 00:00:00+01:00,2019,1,1,0,2.483333,0.5,24,6.0,0.69,22.0,4.4,67.0,957.0,101.0,0.0,2019-01-01,tuesday,holiday,national_holiday,Año Nuevo,1,24,40.417924,-3.670896,24,40.421398,-3.675204,821.854857,0,1ad9b018315170d824bb396889046eaeff8ce0a0be46b3...,1


In [5]:
"""
Test code from stackoverflow https://stackoverflow.com/questions/639695/how-to-convert-latitude-or-longitude-to-meters
Spatial distance blog: https://en.wikipedia.org/wiki/Geographic_coordinate_system#Latitude_and_longitude
Manual test to validate best distance approx.
"""
#Pytagorean distance:
import math

# Define the latitude and longitude of the two points
Lat1 = 40.415010  # Latitude of point 1 Bicimad, 146 Maria Francisca, 3, 28002
Lat2 = 40.444226  # Latitude of point 2 Plaza de la Provincia, Centro, 28012
Lon1 = -3.706103  # Longitude of point 1 Bicimad, 146 Maria Francisca, 3, 28002
Lon2 = -3.678717  # Longitude of point 2 Plaza de la Provincia, Centro, 28012

# Calculate the latitude of the midpoint between the two points
latMid = (Lat1 + Lat2) / 2.0  # or just use Lat1 for a slightly less accurate estimate

# Calculate the number of meters per degree of latitude
m_per_deg_lat = 111132.954 - 559.822 * math.cos(2.0 * latMid) + 1.175 * math.cos(4.0 * latMid)

# Calculate the number of meters per degree of longitude
m_per_deg_lon = (math.pi / 180) * 6367449 * math.cos(latMid)

# Calculate the differences in latitude and longitude
deltaLat = abs(Lat1 - Lat2)
deltaLon = abs(Lon1 - Lon2)

# Calculate the distance in meters using Pythagorean distance
dist_m_pythagoras = math.sqrt(pow(deltaLat * m_per_deg_lat, 2) + pow(deltaLon * m_per_deg_lon, 2))

# Calculate the distance in meters using Manhattan distance
dist_m_manh = abs((Lat1 - Lat2) * m_per_deg_lat) + abs((Lon1 - Lon2)* m_per_deg_lon)

# Output the calculated distance
print(f"Distance between the two points in meters: {dist_m_pythagoras}")
print(f"Manhattan distance between the two points in meters: {dist_m_manh}")
print("Reported Distance in Google Maps in meters: ~5400")

Distance between the two points in meters: 4272.423760950187
Manhattan distance between the two points in meters: 6025.651156393747
Reported Distance in Google Maps in meters: ~5400


In [6]:
"""
Code inspired on stackoverflow https://stackoverflow.com/questions/639695/how-to-convert-latitude-or-longitude-to-meters
Spatial distance blog: https://towardsdatascience.com/spatial-distance-and-machine-learning-2cab72fc6284
"""

# Function to calculate Manhattan distance bewtween latitude longitud coordinates:
def calculate_distance_meters_mh(row, lat_col1, lon_col1, lat_col2, lon_col2):
    Lat1 = row[lat_col1]
    Lon1 = row[lon_col1]
    Lat2 = row[lat_col2]
    Lon2 = row[lon_col2]

    latMid = (Lat1 + Lat2) / 2.0 # Calculate the midpoint between the two latitude points
    m_per_deg_lat = 111132.954 - 559.822 * math.cos(2.0 * latMid) + 1.175 * math.cos(4.0 * latMid) # Calculate the number of meters per degree of latitude
    m_per_deg_lon = (math.pi / 180) * 6367449 * math.cos(latMid) # Calculate the number of meters per degree of longitude
    
    dist_m_manh = abs((Lat1 - Lat2) * m_per_deg_lat) + abs((Lon1 - Lon2) * m_per_deg_lon)

    return dist_m_manh

In [7]:
# Measure approximate distance between unplug and plug BiciMAD station:
labeled_bike_users_df['mh_distance_meters'] = labeled_bike_users_df.apply(calculate_distance_meters_mh, args=('idunplug_latitude', 'idunplug_longitude', 'idplug_latitude', 'idplug_longitude'), axis=1)

In [10]:
labeled_bike_users_df[labeled_bike_users_df['mh_distance_meters'] == labeled_bike_users_df['mh_distance_meters'].max()]

,_id,user_day_code,idplug_base,user_type,idunplug_base,travel_time,idunplug_station,ageRange,idplug_station,unplug_hourTime,zip_code,temp_unplug_hourTime,year,month,day,hour,travel_time_min,travel_revenue,id_weather,80,81,82,83,86,87,88,89,date,week_day,WorkingDay_Holiday_Sunday,holiday_type,holiday_desc,day_of_week,unplug_station_tot_bases,idunplug_latitude,idunplug_longitude,plug_station_tot_bases,idplug_latitude,idplug_longitude,mh_distance_meters,time_group,unique_user_day_code,tot_trips_day
1514395,5ee8ae4989a02fad249eacb3,4d31c4ee0fad7dfb7d9fc9f0f5e1986d1e01cf74056a56...,12,1,1,1453,215,4,187,2019-09-28T01:00:00Z,28001,2019-09-28 01:00:00+00:00,2019,9,28,1,24.216667,0.5,24,NaN,1.05,167.0,20.1,41.0,946.0,225.0,0.0,2019-09-28,saturday,weekend,weekend,weekend,5,18,40.463028,-3.697333,24,40.397972,-3.66925,10067.030477,0,4d31c4ee0fad7dfb7d9fc9f0f5e1986d1e01cf74056a56...,1
1974304,5eeae6f31b91311190d87d9d,2b5f81861c3bb82a1fbb1d1678ad4b9735de94f4ec9b0b...,14,1,1,2659,215,5,187,2019-12-13T19:00:00Z,28029,2019-12-13 19:00:00+00:00,2019,12,13,19,44.316667,1.1,24,NaN,3.59,254.0,12.3,72.0,940.0,60.0,0.0,2019-12-13,friday,workingday,workingday,workingday,4,18,40.463028,-3.697333,24,40.397972,-3.66925,10067.030477,3,2b5f81861c3bb82a1fbb1d1678ad4b9735de94f4ec9b0b...,1


In [8]:
# Function to discretize hours of day based on study https://doi.org/10.1016/j.tra.2018.06.001
def map_hour_day_groups(value):
    """
    Hour day time_group: {0:Night (0:00–7:00) ; 1:AM (7:00–10:00); 2:Midday (10:00–16:00); 3:PM (16:00–20:00); 4:Evening (20:00–24:00)}
    """
    if value >= 7 and value < 10:
        return 1
    elif value >= 10 and value < 16:
        return 2
    elif value >= 16 and value < 20:
        return 3
    elif value >= 20 and value < 24:
        return 4
    else:
        return 0

In [25]:
"""
Hour day time_group: {0:Night (0:00–7:00) ; 1:AM (7:00–10:00); 2:Midday (10:00–16:00); 3:PM (16:00–20:00); 4:Evening (20:00–24:00)}

"""
# Apply map_hour_day_groups function to discretize hour
labeled_bike_users_df['time_group'] = labeled_bike_users_df['hour'].apply(map_hour_day_groups)

# Display the DataFrame
display(labeled_bike_users_df.head())


,_id,user_day_code,idplug_base,user_type,idunplug_base,travel_time,idunplug_station,ageRange,idplug_station,unplug_hourTime,zip_code,temp_unplug_hourTime,year,month,day,hour,travel_time_min,travel_revenue,id_weather,80,81,82,83,86,87,88,89,date,week_day,WorkingDay_Holiday_Sunday,holiday_type,holiday_desc,day_of_week,unplug_station_tot_bases,idunplug_latitude,idunplug_longitude,plug_station_tot_bases,idplug_latitude,idplug_longitude,mh_distance_meters,time_group,unique_user_day_code,tot_trips_day
0,5c4b07ea2f38432e007daaba,a18550d468153bb8e1808d4f0a85955579951832772996...,10,1,1,415,133,5,36,2019-01-01T00:00:00.000+0100,28005,2019-01-01 00:00:00+01:00,2019,1,1,0,6.916667,0.5,24,6.0,0.69,22.0,4.4,67.0,957.0,101.0,0.0,2019-01-01,tuesday,holiday,national_holiday,Año Nuevo,1,24,40.403408,-3.710811,18,40.415010,-3.706103,1760.389172,0,a18550d468153bb8e1808d4f0a85955579951832772996...,1
1,5c4b07ea2f38432e007daabd,5922b4a1a2591161ebf1ad66f42f52cc7796f3754c2c76...,18,1,6,417,105,3,155,2019-01-01T00:00:00.000+0100,28009,2019-01-01 00:00:00+01:00,2019,1,1,0,6.950000,0.5,24,6.0,0.69,22.0,4.4,67.0,957.0,101.0,0.0,2019-01-01,tuesday,holiday,national_holiday,Año Nuevo,1,24,40.424846,-3.673863,18,40.444226,-3.678717,2641.794256,0,5922b4a1a2591161ebf1ad66f42f52cc7796f3754c2c76...,1
2,5c4b07ea2f38432e007daabe,ed1c1a22c120e7d42cc522c33ab41db549791f3914b273...,4,1,12,448,51,4,44,2019-01-01T00:00:00.000+0100,28005,2019-01-01 00:00:00+01:00,2019,1,1,0,7.466667,0.5,24,6.0,0.69,22.0,4.4,67.0,957.0,101.0,0.0,2019-01-01,tuesday,holiday,national_holiday,Año Nuevo,1,24,40.404785,-3.702827,24,40.413885,-3.704941,1221.373443,0,ed1c1a22c120e7d42cc522c33ab41db549791f3914b273...,1
3,5c4b07ea2f38432e007daac2,b70eb1a71e53c6351b4835f618a459886e2045a44635e4...,17,1,11,468,100,5,171,2019-01-01T00:00:00.000+0100,28028,2019-01-01 00:00:00+01:00,2019,1,1,0,7.800000,0.5,24,6.0,0.69,22.0,4.4,67.0,957.0,101.0,0.0,2019-01-01,tuesday,holiday,national_holiday,Año Nuevo,1,24,40.422658,-3.687055,24,40.429887,-3.671282,2405.058377,0,b70eb1a71e53c6351b4835f618a459886e2045a44635e4...,1
4,5c4b07ea2f38432e007daac4,1ad9b018315170d824bb396889046eaeff8ce0a0be46b3...,13,1,2,149,67,4,65,2019-01-01T00:00:00.000+0100,28009,2019-01-01 00:00:00+01:00,2019,1,1,0,2.483333,0.5,24,6.0,0.69,22.0,4.4,67.0,957.0,101.0,0.0,2019-01-01,tuesday,holiday,national_holiday,Año Nuevo,1,24,40.417924,-3.670896,24,40.421398,-3.675204,821.854857,0,1ad9b018315170d824bb396889046eaeff8ce0a0be46b3...,1


In [26]:
# It seems there are some user_day_code being used in multiple days
# We will create a unique code combining user_day_code together with day col
labeled_bike_users_df['unique_user_day_code'] = labeled_bike_users_df['user_day_code'].astype(str) + '_' + labeled_bike_users_df['day'].astype(str) 

# Calculate total number of trips performed by a user in a day
labeled_bike_users_df['tot_trips_day'] = labeled_bike_users_df['unique_user_day_code'].map(labeled_bike_users_df['unique_user_day_code'].value_counts())
display(labeled_bike_users_df.head())

,_id,user_day_code,idplug_base,user_type,idunplug_base,travel_time,idunplug_station,ageRange,idplug_station,unplug_hourTime,zip_code,temp_unplug_hourTime,year,month,day,hour,travel_time_min,travel_revenue,id_weather,80,81,82,83,86,87,88,89,date,week_day,WorkingDay_Holiday_Sunday,holiday_type,holiday_desc,day_of_week,unplug_station_tot_bases,idunplug_latitude,idunplug_longitude,plug_station_tot_bases,idplug_latitude,idplug_longitude,mh_distance_meters,time_group,unique_user_day_code,tot_trips_day
0,5c4b07ea2f38432e007daaba,a18550d468153bb8e1808d4f0a85955579951832772996...,10,1,1,415,133,5,36,2019-01-01T00:00:00.000+0100,28005,2019-01-01 00:00:00+01:00,2019,1,1,0,6.916667,0.5,24,6.0,0.69,22.0,4.4,67.0,957.0,101.0,0.0,2019-01-01,tuesday,holiday,national_holiday,Año Nuevo,1,24,40.403408,-3.710811,18,40.415010,-3.706103,1760.389172,0,a18550d468153bb8e1808d4f0a85955579951832772996...,1
1,5c4b07ea2f38432e007daabd,5922b4a1a2591161ebf1ad66f42f52cc7796f3754c2c76...,18,1,6,417,105,3,155,2019-01-01T00:00:00.000+0100,28009,2019-01-01 00:00:00+01:00,2019,1,1,0,6.950000,0.5,24,6.0,0.69,22.0,4.4,67.0,957.0,101.0,0.0,2019-01-01,tuesday,holiday,national_holiday,Año Nuevo,1,24,40.424846,-3.673863,18,40.444226,-3.678717,2641.794256,0,5922b4a1a2591161ebf1ad66f42f52cc7796f3754c2c76...,1
2,5c4b07ea2f38432e007daabe,ed1c1a22c120e7d42cc522c33ab41db549791f3914b273...,4,1,12,448,51,4,44,2019-01-01T00:00:00.000+0100,28005,2019-01-01 00:00:00+01:00,2019,1,1,0,7.466667,0.5,24,6.0,0.69,22.0,4.4,67.0,957.0,101.0,0.0,2019-01-01,tuesday,holiday,national_holiday,Año Nuevo,1,24,40.404785,-3.702827,24,40.413885,-3.704941,1221.373443,0,ed1c1a22c120e7d42cc522c33ab41db549791f3914b273...,1
3,5c4b07ea2f38432e007daac2,b70eb1a71e53c6351b4835f618a459886e2045a44635e4...,17,1,11,468,100,5,171,2019-01-01T00:00:00.000+0100,28028,2019-01-01 00:00:00+01:00,2019,1,1,0,7.800000,0.5,24,6.0,0.69,22.0,4.4,67.0,957.0,101.0,0.0,2019-01-01,tuesday,holiday,national_holiday,Año Nuevo,1,24,40.422658,-3.687055,24,40.429887,-3.671282,2405.058377,0,b70eb1a71e53c6351b4835f618a459886e2045a44635e4...,1
4,5c4b07ea2f38432e007daac4,1ad9b018315170d824bb396889046eaeff8ce0a0be46b3...,13,1,2,149,67,4,65,2019-01-01T00:00:00.000+0100,28009,2019-01-01 00:00:00+01:00,2019,1,1,0,2.483333,0.5,24,6.0,0.69,22.0,4.4,67.0,957.0,101.0,0.0,2019-01-01,tuesday,holiday,national_holiday,Año Nuevo,1,24,40.417924,-3.670896,24,40.421398,-3.675204,821.854857,0,1ad9b018315170d824bb396889046eaeff8ce0a0be46b3...,1


In [27]:
# Remove invalid values: travel_time = 0 and Manhattan distance >500 meters (3 Instances) 
display(labeled_bike_users_df[(labeled_bike_users_df['travel_time']==0)&(labeled_bike_users_df['mh_distance_meters']>500)])
print(f"Full labeled bike users dataset: {labeled_bike_users_df.shape}")

labeled_bike_users_df =  labeled_bike_users_df.loc[(labeled_bike_users_df['travel_time'] != 0) | (labeled_bike_users_df['mh_distance_meters'] <= 500)]

print(f"labeled bike users dataset size w/0 invalid travel time + mh distance: {labeled_bike_users_df.shape}")

,_id,user_day_code,idplug_base,user_type,idunplug_base,travel_time,idunplug_station,ageRange,idplug_station,unplug_hourTime,zip_code,temp_unplug_hourTime,year,month,day,hour,travel_time_min,travel_revenue,id_weather,80,81,82,83,86,87,88,89,date,week_day,WorkingDay_Holiday_Sunday,holiday_type,holiday_desc,day_of_week,unplug_station_tot_bases,idunplug_latitude,idunplug_longitude,plug_station_tot_bases,idplug_latitude,idplug_longitude,mh_distance_meters,time_group,unique_user_day_code,tot_trips_day
528746,5cabc4db2f384315f4fec9e0,31a3ebdf1ae0e4da67021f4d0ccfd4214461b3a5aa6ffc...,7,1,20,0,123,4,64,2019-04-04T20:00:00.000+0200,28001,2019-04-04 20:00:00+02:00,2019,4,4,20,0.0,0.0,24,30.0,2.06,237.0,8.4,36.0,932.0,268.0,0.0,2019-04-04,thursday,workingday,workingday,workingday,3,24,40.424020,-3.711603,24,40.419752,-3.688398,2828.667086,4,31a3ebdf1ae0e4da67021f4d0ccfd4214461b3a5aa6ffc...,2
992159,5d0ab0c22f38432ef80d7b76,06430323de835713776359fa34074366b64c0e7729c40e...,6,1,3,0,155,4,116,2019-06-15T15:00:00.000+0200,28540,2019-06-15 15:00:00+02:00,2019,6,15,15,0.0,0.0,24,NaN,1.90,47.0,18.8,35.0,945.0,369.0,0.0,2019-06-15,saturday,weekend,weekend,weekend,5,18,40.444226,-3.678717,24,40.434497,-3.720689,5371.277624,2,06430323de835713776359fa34074366b64c0e7729c40e...,6
992390,5d0ab0c7a9cd6441ac480ea1,06430323de835713776359fa34074366b64c0e7729c40e...,6,1,3,0,155,4,116,2019-06-15T15:00:00.000+0200,28540,2019-06-15 15:00:00+02:00,2019,6,15,15,0.0,0.0,24,NaN,1.90,47.0,18.8,35.0,945.0,369.0,0.0,2019-06-15,saturday,weekend,weekend,weekend,5,18,40.444226,-3.678717,24,40.434497,-3.720689,5371.277624,2,06430323de835713776359fa34074366b64c0e7729c40e...,6


Full labeled bike users dataset: (2035397, 43)
labeled bike users dataset size w/0 invalid travel time + mh distance: (2035394, 43)


In [28]:
labeled_bike_users_df.shape

(2035394, 43)

In [29]:
# Export final dataset including feature creation after merging all datasets and feature creation
labeled_bike_users_df.to_csv('add_name.csv', index=False, sep = ';')